<div class="alert alert-block alert-info"><b></b>
<h1><center> <font color='black'> Homework 02  </font></center></h1>
<h2><center> <font color='black'> Customer segmentation and regression problems </font></center></h2>    
<h2><center> <font color='black'> BDA - University of Tartu - Spring 2020</font></center></h3>
</div>

# Homework instructions

- Insert your team member names and student IDs in the field "Team mates" below. If you are not working in a team please insert only your name, surname and student ID 

- The accepted submission formats are Colab links or .ipynb files. If you are submitting Colab links please make sure that the privacy settings for the file is public so we can access your code. 

- The submission will automatically close at 12:00 am, so please make sure you have enough time to submit the homework. 

- Only one of the teammates should submit the homework. We will grade and give points to both of you! 

- You do not necessarily need to work on Colab. Especially as the size and the complexity of datasets will increase through the course, you can install jupyter notebooks locally and work from there.

- If you do not understand what a question is asking for, please ask in Moodle. 


**<h2><font color='red'>Team mates:</font></h2>**


<font color='red'>Name Surname: Enlik -</font>&emsp;   <font color='red'>Student ID: B96323</font>


<font color='red'>Name Surname: XXXXX</font>&emsp;   <font color='red'>Student ID: YYYY</font>




In [1]:
import numpy as np 
import pandas as pd 
import datetime as dt 

# 1. RFM technique (3 points)

In this homework we will apply the RFM technique similarly to what we did in the practice session. The dataset we are using belongs to http://donorschoose.org/. 

The web platform http://donorschoose.org helps projects in collecting donations. The donators do have the option to give 15% of what they donate to this webplatform. The dataset contains the following fields:

- Donor ID - Unique identifier of donors
- Donation Included Optional Donation - The donor has agreed(Yes/No) to give 15% of donation amount to Donoschoose.org
- Donation Amount - Total amount donated
- Donor Cart Sequence - The ranking of the donaton project in the donors list
- Donation Received Date - Date of online payment


**1.0.**
**Please load the dataset Donations.csv. The field "Donation Included Optional Donation" shows if the donor has agreed (Yes/No) to give 15% of donation amount to Donoschoose.org. Update the field "Donation Amount" as such that it will show the correct amount the donor has given for the charity projects in case where the answer is Yes to "Donation Included Optional Donation". (0.30 points)** 

In [30]:
donations = pd.read_csv("Donations.csv", sep = " ")

In [31]:
donations.head()

,Donor ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date
0,1f4b5b6e68445c6c4a0509b3aca93f38,No,178.37,11,2016-08-23 13:15:57
1,bd323208dc78b1c74b62664b768f3176,Yes,200.00,2,2013-02-17 21:36:24
2,6dd6113f89f2766d3b0707ef2a46260c,Yes,10.00,44,2013-02-27 10:32:22
3,344ad0a72366a27bd4926bbe5d066939,Yes,10.00,9,2017-07-24 08:40:35
4,3cf2607d4131cd654bf54a1f5a533357,Yes,25.00,5,2017-07-11 16:40:30


In [32]:
def updateDonationAmount(isIncluded, amount):
    if isIncluded == "Yes":
        return amount - (0.15 * amount)
    else:
        return amount

donations['Donation Amount'] = donations.apply(
    lambda x: updateDonationAmount(x['Donation Included Optional Donation'], x['Donation Amount']), axis=1)

In [33]:
donations.head()

,Donor ID,Donation Included Optional Donation,Donation Amount,Donor Cart Sequence,Donation Received Date
0,1f4b5b6e68445c6c4a0509b3aca93f38,No,178.37,11,2016-08-23 13:15:57
1,bd323208dc78b1c74b62664b768f3176,Yes,170.00,2,2013-02-17 21:36:24
2,6dd6113f89f2766d3b0707ef2a46260c,Yes,8.50,44,2013-02-27 10:32:22
3,344ad0a72366a27bd4926bbe5d066939,Yes,8.50,9,2017-07-24 08:40:35
4,3cf2607d4131cd654bf54a1f5a533357,Yes,21.25,5,2017-07-11 16:40:30


**1.1. For `Recency`, consider 2018-05-31 23:00:00 as the reference date
(we assume this is report date). Calculate the number of days between this date and the date of last purchase  for each customer. How long the donor with ID equal to e0fe4d9b8def8a71635e65ba4ff5ef40 has been inactive ? (0.30 points)** 


In [37]:
# Reference
# https://stackoverflow.com/questions/50089903/convert-column-to-timestamp-pandas-dataframe

# Convert column 'Donation Received Date' into date type (timestamp)
donations['Donation Received Date'] = pd.to_datetime(donations['Donation Received Date'], format = "%Y-%m-%d %H:%M:%S")

In [76]:
# Create a generic user dataframe to keep Donor ID and new segmentation scores (RFM)
donations_user = pd.DataFrame(donations['Donor ID'].unique())
donations_user.columns = ['Donor ID']
print(donations_user)


                                Donor ID
0       1f4b5b6e68445c6c4a0509b3aca93f38
1       bd323208dc78b1c74b62664b768f3176
2       6dd6113f89f2766d3b0707ef2a46260c
3       344ad0a72366a27bd4926bbe5d066939
4       3cf2607d4131cd654bf54a1f5a533357
...                                  ...
129060  7c3a054108a9eebb441054450d76a9d3
129061  da71310482043b4da3e1bb1ab80850e8
129062  d1e1e4fc0975a7936940a3708f1860c9
129063  ead077ee750c0a4d2190afdb202a6343
129064  56729cfddae7e622f7d49ba5877c4e5c

[129065 rows x 1 columns]


In [60]:
# Get the last purchase date for each donors and create a dataframe from it 
lastPurchaseDate = donations.groupby('Donor ID')['Donation Received Date'].max().reset_index()
lastPurchaseDate.columns = ['Donor ID', 'LastPurchaseDate']
print(lastPurchaseDate)

                                Donor ID    LastPurchaseDate
0       00002d44003ed46b066607c5455a999a 2017-10-18 14:34:11
1       00002eb25d60a09c318efbd0797bffb5 2018-01-16 15:32:41
2       00006084c3d92d904a22e0a70f5c119a 2018-04-14 23:04:42
3       0001107b9faa5c3bb42cfcecece1d587 2018-03-24 19:43:59
4       0001f63e9437ebbba6ddaae0664037a7 2017-03-09 07:10:03
...                                  ...                 ...
129060  fffc8063fefd271719cb27cd019de746 2016-09-01 22:40:57
129061  fffcffed2771863a61c9669049cc6876 2013-11-25 23:49:34
129062  fffd67203dc580485e30d8a8820ef2bf 2018-04-25 18:50:51
129063  fffd69c0f1dc11f7fc10d595891723a2 2018-03-15 00:38:54
129064  fffe56ecd1382490b5995a308d9476d9 2015-08-05 17:25:37

[129065 rows x 2 columns]


In [75]:
# create variable reportDate based on mentioned reference date above
reportDate = dt.datetime.strptime('2018-05-31 23:00:00', '%Y-%m-%d %H:%M:%S')

lastPurchaseDate['Recency'] = (reportDate - lastPurchaseDate['LastPurchaseDate']).dt.days
print(lastPurchaseDate)

                                Donor ID    LastPurchaseDate  Recency
0       00002d44003ed46b066607c5455a999a 2017-10-18 14:34:11      225
1       00002eb25d60a09c318efbd0797bffb5 2018-01-16 15:32:41      135
2       00006084c3d92d904a22e0a70f5c119a 2018-04-14 23:04:42       46
3       0001107b9faa5c3bb42cfcecece1d587 2018-03-24 19:43:59       68
4       0001f63e9437ebbba6ddaae0664037a7 2017-03-09 07:10:03      448
...                                  ...                 ...      ...
129060  fffc8063fefd271719cb27cd019de746 2016-09-01 22:40:57      637
129061  fffcffed2771863a61c9669049cc6876 2013-11-25 23:49:34     1647
129062  fffd67203dc580485e30d8a8820ef2bf 2018-04-25 18:50:51       36
129063  fffd69c0f1dc11f7fc10d595891723a2 2018-03-15 00:38:54       77
129064  fffe56ecd1382490b5995a308d9476d9 2015-08-05 17:25:37     1030

[129065 rows x 3 columns]


In [77]:
#
donations_user = pd.merge(donations_user, lastPurchaseDate[['Donor ID','Recency']], on='Donor ID')
print(donations_user)

                                Donor ID  Recency
0       1f4b5b6e68445c6c4a0509b3aca93f38      126
1       bd323208dc78b1c74b62664b768f3176     1012
2       6dd6113f89f2766d3b0707ef2a46260c     1025
3       344ad0a72366a27bd4926bbe5d066939      242
4       3cf2607d4131cd654bf54a1f5a533357      133
...                                  ...      ...
129060  7c3a054108a9eebb441054450d76a9d3       93
129061  da71310482043b4da3e1bb1ab80850e8     1871
129062  d1e1e4fc0975a7936940a3708f1860c9      498
129063  ead077ee750c0a4d2190afdb202a6343      289
129064  56729cfddae7e622f7d49ba5877c4e5c     1668

[129065 rows x 2 columns]


In [85]:
# plot a recency histogram distribution of recency across our donors.
# reference: BDA2020 - lab03 practice material
# https://drive.google.com/file/d/1bTAKH9MOyf3Wg5qZ3nrjNAqD_plwGt4H/view

import plotly.offline as pyoff
import plotly.graph_objs as go

plot_data = [
    go.Histogram(
        x=donations_user['Recency']
    )
]

plot_layout = go.Layout(
        title='Distribution of recency across our Donor.'
    )

fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [84]:
donations_user[donations_user['Donor ID'] == "e0fe4d9b8def8a71635e65ba4ff5ef40"]

,Donor ID,Recency
67193,e0fe4d9b8def8a71635e65ba4ff5ef40,34


**<font color='red'>Answer:</font>** 34 days

**1.2. For the "Frequency" calculate how many times  a user has donated. What is the frequency value for the donor with ID e0fe4d9b8def8a71635e65ba4ff5ef40? (0.30 points)**

**<font color='red'>Answer:</font>** 

**1.3. For the "Revenue" calculate how much a donor has donated. What is the revenue value for the donor with ID e0fe4d9b8def8a71635e65ba4ff5ef40? (0.30 points)**

**<font color='red'>Answer:</font>**

**1.4. Use the quantiles to separate Recency, Frequency, Revenue into four bins. We define our high value customers which have low Recency value and high Frequency and Revenue values. Calculate score for each donor which is equal to sum of bin values as a RFM components and save it in a new column. (0.50 points)**

**1.5. Build a 3D graph where axes represent the R, F and, M values and the hue (color of shade) represents the score. Based on the graph on how many categories would you cluster the donors? Please explain your decision. (0.50 points)**

**<font color='red'>Answer:</font>** 

**1.6. Interpret what the minimum, maximum and mean score indicates about your donors ?(0.50 points)**

**<font color='red'>Answer:</font>** 

**1.7. Based on your answer in the question 1.5, please threshold the scores to create as many clusters as you think is reasonable. Assign integer values to those clusters to ease the identification and add a new column to your RFM data frame which shows the cluster each record belongs to. (0.30 points)**

# 2. Automated clustering algorithms (4.1 points)

**2.1. Use K-Means algorithm with parameter "max_iter = 1", to get the same number of clusters for RFM as you selected in the exercise 1.5.  Feed the RFM values at the same time to the algorithm (not separately). Store your result in a new column and name this column as Kmeans_cluster.(0.50 points)**

**2.2. Take a look at the cluster centers given by KMeans after prediction. Are they identical to mean value of each cluster ? If yes, why do you think so? If no, why do you think so? In case the answer is no, please try to change the parameters of the function so the cluster center and the mean value of clusters will be identical. Store and report the new results. (0.75 points)**

**<font color='red'>Answer:</font>** 

**2.3. Compare the results you produced in exercise 1.7 with the results generated by KMeans i.e number of elements in each class, mean and std values for elements in each class. Which approach do you think is better or are they similar ? Hint: Plot histogram and/or boxplot (0.75 points)**

**<font color='red'>Answer:</font>**

**2.4. To overcome the limitations of KMeans many other clustering algorithms are used, one of them is DBSCAN. Mention at least 2 advantages of DBSCAN over KMeans and 2 disadvantages of DBSCAN.(0.30 points)**

**<font color='red'>Answer:</font>**

**2.5. Use DBSCAN  with its default parameters to generate clusters based on RFM values. How many clusters did it create? Was the value same as you selected in the exercise 1.5?(0.50 points)**

**2.6. In 2.5, how many noisy data points were identified by the DBSCAN? (0.30 points)**

**<font color='red'>Answer:</font>**

**2.7 Try different combinations of "eps" and "min_samples" parameters for DBSCAN in order to reach the same number of classes as in exercise 1.5 (at most you can have a difference of 10 classes). Report the combination that gave the best result (0.75 points). Why do you think it worked and the number of classes decreased compared to exercise 2.5? (0.25 points)** 

**<font color='red'>Answer:</font>**

# 3. Regression (2.9 points)



For this exercise we are going to use the Prices dataset which contains 74 columns. Each column explains a  characteristic of houses in sale. The last column is describing their prices. 

In [0]:
data = pd.read_csv("Prices.csv")
data.head()

Before using the data we have to apply some preprocessing:
- Apply one-hot encoding to categorical data types
- Apply logarithmic transformation to the data
- Replace negative infinite values with 0

In [0]:
X = data.iloc[:, 0:-1]
y = data.iloc[:, -1]

data = pd.get_dummies(X)
data["SalePrice"]=y

data = np.log(data)

data[data==-np.inf]=0

Now the data is ready for you to use in the next exercises.

**3.1. Calculate the correlation between price and each feature. Which are the top 3 features that have the highest correlation with  price? Is the correlation positive or negative? Explain what happens with the price when each of 
those 3 features change (consider only one feature at a time) and others are kept constant. (0.50 points)** 



<font color='red'> **Answer:**</font>


**3.2.  In this exercise we have to build  a regression model using training data and then predict the price in test data. You are free to select features which you want to use for building the model. As the data has missing values we are asking you to try the following methods for dealing with the missing data:**

a) mean imputation

b) median imputation

c) mode imputation

d) dropping missing values


**For each of these cases report MAE, RMSE and R<sup>2</sup>. Which method works better ?**

**To get the training and test set, split  the data into 20% test set and 80% training set using train_test_split function from scikit-learn. Keep the parameter random_state equal to 0. (1.50 points)**

In [0]:
<font color='red'> **Answer:**</font>


In [0]:
## Place here the values for MAE, RMSE, R^2 received from the best method
mae_best = ...
rmse_best = ...
r2_best = ...

**3.3 From 3.2 keep the best method to deal with missing values and use PCA to reduce the number of features to 34 components. Keep random_state for PCA equal to 0. (0.30 points)**

**3.4. What percentage of the variance is explained by the first component ? (0.30 points)**

<font color='red'> **Answer:**</font>


**3.5. Use the new components derived from PCA to predict the house pricing. Keep the ratio of test and train set to 20/80 and the random_state equal to 0. Report MAE, RMSE and R<sup>2</sup> (0.30 points)**

In [0]:
## Calculate the values for MAE, RMSE, R^2 received after applying PCA

mae_pca = ...
rmse_pca = ...
r2_pca = ...


In [0]:
print("MAE difference after PCA: ", mae_best-mae_pca)
print("RMSE difference after PCA: ", rmse_best-rmse_pca)
print("R2 difference after PCA: ", r2_best-r2_pca)

## How long did it take you to solve the homework?

* Please answer as precisely as you can. It does not affect your points or grade in any way. It is okay, if it took 0.5 hours or 24 hours. The collected information will be used to improve future homeworks.

<font color='red'> **Answer:**</font>

**<font color='red'>(please change X in the next cell into your estimate)</font>**

X hours

## What is the level of difficulty for this homework?
you can put only number between $0:10$ ($0:$ easy, $10:$ difficult)

<font color='red'> **Answer:**</font>